In [ ]:
from datetime import datetime
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

import statsmodels.api as sm
import warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [ ]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define the p, d, and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q, and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q, and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
temp_model = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        
        try:
            temp_model = SARIMAX(train_data['Harga Beras'],
                                 order = param,
                                 seasonal_order = param_seasonal,
                                 exog = train_data.drop('Harga Beras', axis=1),
                                 enforce_stationarity=False,
                                 enforce_invertibility=False)
            results = temp_model.fit(maxiter=100)

            # print("SARIMAX{}x{}12 - AIC:{}".format(param, param_seasonal, results.aic))
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            continue
print("Best SARIMAX{}x{}12 model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming dataset is your DataFrame and it has been preprocessed

# Split the data into training and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, shuffle=False)

# Define the model
model = SARIMAX(dataset['Harga Beras'], 
                order=(0, 1, 1), 
                seasonal_order=(0, 1, 1, 12), 
                exog=dataset.drop('Harga Beras', axis=1))

# Fit the model
model_fit = model.fit(disp=False)

# Make prediction for the next 11 steps
exog_future = dataset.drop('Harga Beras', axis=1).iloc[-11:]  # Get last 11 rows of exogenous data
forecast = model_fit.predict(len(dataset), len(dataset) + 10, exog=exog_future)

print(forecast)

In [ ]:
# Fit the model on the training data
model_fit = model.fit(disp=False)

# Make prediction for the same number of steps as the size of your test data
exog_future = test_data.drop('Harga Beras', axis=1)  # Use exogenous data from the test set
forecast = model_fit.predict(len(train_data), len(train_data) + len(test_data) - 1, exog=exog_future)

# Now the test data and forecast should have the same number of samples
assert len(test_data['Harga Beras']) == len(forecast), "Number of samples does not match"

# Calculate the error metrics
mae = mean_absolute_error(test_data['Harga Beras'], forecast)
mse = mean_squared_error(test_data['Harga Beras'], forecast)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual values
plt.plot(test_data.index, test_data['Harga Beras'], label='Actual')

# Plot the predicted values
plt.plot(test_data.index, forecast, label='Predicted')

plt.title('Actual vs Predicted Values')
plt.xlabel('Date')
plt.ylabel('Harga Beras')
plt.legend()

plt.show()

In [ ]:
# Fit the model on the entire data
model_fit = model.fit(disp=False)

# Make forecast for the next 90 days
# Assume that the future values of the exogenous variables will be the same as their most recent values
exog_future = pd.DataFrame([dataset.drop('Harga Beras', axis=1).iloc[-1]] * 90)

forecast_results = model_fit.get_forecast(steps=90, exog=exog_future)

# Get the forecast and the confidence interval
forecast = forecast_results.predicted_mean
conf_int = forecast_results.conf_int()

# Plot the actual values
plt.plot(dataset.index, dataset['Harga Beras'], label='Actual')

# Plot the forecasted values
forecast_index = pd.date_range(start='2024-01-01', end='2024-03-30', freq='D')  # Create a date range for the forecast
plt.plot(forecast_index, forecast, label='Forecast')

# Plot the confidence interval
plt.fill_between(forecast_index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='grey', alpha=0.3)

plt.title('Actual vs Forecasted Values')
plt.xlabel('Date')
plt.ylabel('Harga Beras')
plt.legend()

plt.show()

# Get the first and last date in the dataset
start_date = dataset.index.min()
end_date = dataset.index.max()

# Get the in-sample prediction and the confidence interval
in_sample_preds = model_fit.get_prediction(start=start_date, end=end_date, dynamic=False)
in_sample = in_sample_preds.predicted_mean

# Calculate the errors for the in-sample prediction
mse = mean_squared_error(dataset['Harga Beras'], in_sample)
rmse = np.sqrt(mse)
mae = mean_absolute_error(dataset['Harga Beras'], in_sample)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')